In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras import layers, models, metrics, optimizers, losses, callbacks

from utils.data_loader import create_dataset
from utils.model_loss import TverskyBCEPerSequence
from utils.model_inference_plots import *

In [ ]:
df = pd.read_parquet('data/final_model_data_all_scaled.parquet')

In [ ]:
X = df[['Bx', 'By', 'Bz', 'Bx_lag_1', 'Bx_lag_2', 'By_lag_1',
        'By_lag_2', 'Bz_lag_1', 'Bz_lag_2', 'Bx_conditional_vol',
        'By_conditional_vol', 'Bz_conditional_vol', 'Bx_rolling_stdev',
        'By_rolling_stdev', 'Bz_rolling_stdev']].values

y = df['Event_label_80'].values

In [ ]:
# Data is split manually since it was randomly shuffled in 6-3_model_dataset.ipynb
total_samples = len(X)
n_features = X.shape[1]

# 80% of the data used for training
# for models with just one satellite 60% of the data was used for training
train_size = int(0.8 * total_samples)
test_size = total_samples - train_size

In [ ]:
# Batch size for the model, n_timesteps is the number of timesteps in each sequence
# 500 corresponds to ~25 minutes of data which is the average event duration
# The stride of 40 reduces redundancy between sequences and speeds up training times
batch_size = 256
n_timesteps = 500
stride = 40

train_idx = (0, train_size)
test_idx = (train_size, total_samples)

# Creates the train and test datasets for the model
train_dataset = create_dataset(X, y, n_timesteps, batch_size, stride, train_idx[0], train_idx[1])
test_dataset = create_dataset(X, y, n_timesteps, batch_size, stride, test_idx[0], test_idx[1])

In [ ]:
# Determines the number of steps to train and test the model, if not defined model
# will run indefinitely 
steps_train_epoch = int(np.ceil((train_size - n_timesteps) / (stride * batch_size)))
steps_test_epoch = int(np.ceil((test_size - n_timesteps) / (stride * batch_size)))

In [ ]:
# This cell determines the optimal number of batches by looking at the 
# distribution of sequences with events

batch_ratios = []

for _, outputs in test_dataset.take(steps_train_epoch):
    y_time = outputs['time_output'].numpy()
    has_event = (np.sum(y_time, axis=1) > 0).astype(np.float32)
    
    batch_ratios.append(np.mean(has_event))

plt.hist(batch_ratios, bins=30, edgecolor='black')
plt.xlabel("Percentage of 1s in Sequence")
plt.ylabel("Count")
plt.title("Distribution of Sequences with 1s per Batch (Train Set)")
plt.show()

In [ ]:
# Set random seed for reproducibility
tf.random.set_seed(42)

# Input shape: (batch_size, n_timesteps, n_features)
input_layer = layers.Input(shape=(n_timesteps, n_features))

# Initial convolution to capture local patterns in the time series
x = layers.Conv1D(kernel_size=5, filters=64, padding='same', activation='gelu')(input_layer)
x = layers.LayerNormalization()(x)

# First BiLSTM layer to capture temporal dependencies in both directions
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)

# Self-attention layer to allow each time step to attend to others
attention, attention_weights = layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x, return_attention_scores=True)

# Residual connection + normalization
x = layers.Add()([x, attention])
x = layers.LayerNormalization()(x)

# Second BiLSTM layer for deeper sequence modeling
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)

# Save this intermediate output for a skip connection
skip = x

# Feedforward layers to transform representation
x = layers.Dense(128, activation='gelu')(x)
x = layers.Dense(64, activation='gelu')(x)

# Down-project skip connection to match dimensionality
skip = layers.Dense(64)(skip)

# Concatenate skip connection with feedforward output
x = layers.Concatenate()([x, skip])

# Final transformation before output layers
x = layers.Dense(32, activation='gelu')(x)

# Time-distributed output (per time step): sigmoid for binary classification
output_time_layer = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'), name="time_output")(x)

# Sequence-level output: Global average pooling over the features to capture latent patterns, then sigmoid
# The model learns to relate different event densities to different patterns
x_seq = layers.GlobalAveragePooling1D()(x)
output_seq_layer = layers.Dense(1, activation='sigmoid', name="sequence_output")(x_seq)

# Define the full model with two outputs
model = models.Model(inputs=input_layer, outputs=[output_time_layer, output_seq_layer])

# Compile the model with custom and standard loss functions
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    
    # Custom Tversky + Focal BCE loss for time step predictions,
    # and Huber loss for sequence-level summary prediction
    loss={
        'time_output': TverskyBCEPerSequence(
            alpha_t=0.6,
            beta_t=0.7,
            alpha_f=0.25,
            gamma_f=1.5,
            event_weight=1.75
        ),
        'sequence_output': losses.Huber()
    },
    
    # Equal weighting for both losses
    loss_weights={
        'time_output': 1.0,
        'sequence_output': 1.0
    },
    
    # Track accuracy, precision, and recall for the time step predictions
    metrics={
        'time_output': ['accuracy', metrics.Precision(), metrics.Recall()]
    }
)

In [ ]:
lr_schedule = callbacks.ReduceLROnPlateau(
    monitor='loss',
    factor=0.5,
    patience=2,
    verbose=0,
    min_lr=1e-6
)

In [ ]:
model.fit(
    train_dataset,
    epochs=10,
    steps_per_epoch=steps_train_epoch,
    callbacks=[lr_schedule],
    verbose=1
)

In [ ]:
# keras.utils.plot_model(
#     model,
#     to_file="model.png",
#     show_shapes=True,
#     show_dtype=False,
#     show_layer_names=False,
#     rankdir="TD",
#     expand_nested=False,
#     dpi=200,
#     show_layer_activations=False,
#     show_trainable=False,
# )

In [ ]:
model.save("models/mosrl_80_all_model.keras")

In [ ]:
y_pred_probas_raw = model.predict(test_dataset, steps=steps_test_epoch, verbose=1)

In [ ]:
# extract the per timestep predictions
y_pred_probas_sqzd = y_pred_probas_raw[0].squeeze(-1)
num_windows, window_size = y_pred_probas_sqzd.shape

# Calculate the total output length to align predictions with the original time series
# Subtracts 39 to align with y_test length (if needed). Remove "- 39" if not applicable.
output_len = num_windows * stride + window_size - 39

# Initialize arrays to store the sum of predictions and counts for averaging
sum_preds = np.zeros(output_len, dtype=y_pred_probas_sqzd.dtype)
count_preds = np.zeros(output_len, dtype=int)

# Loop over each sliding window prediction
for win_num in range(num_windows):
    start = win_num * stride # Start index of the window in the original timeline
    end = start + window_size # End index of the window
    
    # Accumulate the predictions from overlapping windows
    sum_preds[start:end] += y_pred_probas_sqzd[win_num]
    
    # Count how many times each time step has been predicted
    count_preds[start:end] += 1

# Average the predictions across overlapping windows (ignoring zero divisions)
y_pred_probas = np.divide(sum_preds, count_preds, where=count_preds != 0)

In [ ]:
np.save('models/mosrl_80_all_pred_probas.npy', y_pred_probas)